# Read SIFT 1 million 

In [1]:
using HDF5
using BenchmarkTools
using Distances
using LoopVectorization
using SIMD
using Clustering
using ProgressMeter
using StaticArrays

In [2]:
Sys.cpu_info()[1].model

"Apple M1 Pro"

In [3]:
path = joinpath(homedir(), "Datasets", "SIFT1M",
                "sift-128-euclidean.hdf5")

"/Users/dbuchaca/Datasets/SIFT1M/sift-128-euclidean.hdf5"

In [4]:
f = h5open(path, "r")

🗂️ HDF5.File: (read-only) /Users/dbuchaca/Datasets/SIFT1M/sift-128-euclidean.hdf5
├─ 🏷️ distance
├─ 🔢 distances
├─ 🔢 neighbors
├─ 🔢 test
└─ 🔢 train

In [5]:
X_tr_vecs = read(f["train"])
X_te_vecs = read(f["test"]);
neighbors = read(f["neighbors"])
distances = read(f["distances"])

@show size(X_tr_vecs)
@show size(X_te_vecs)
@show size(neighbors)
@show size(distances)

size(X_tr_vecs) = (128, 1000000)
size(X_te_vecs) = (128, 10000)
size(neighbors) = (100, 10000)
size(distances) = (100, 10000)


(100, 10000)

## Clustering.jl

Create the pqcodes for the training points

In [6]:
n_features = size(X_tr_vecs,1)
n_features

128

In [7]:
unique_values = sort(unique(X_tr_vecs))

213-element Vector{Float32}:
   0.0
   1.0
   2.0
   3.0
   4.0
   5.0
   6.0
   7.0
   8.0
   9.0
  10.0
  11.0
  12.0
   ⋮
 202.0
 204.0
 205.0
 206.0
 207.0
 208.0
 210.0
 211.0
 212.0
 214.0
 216.0
 218.0

In [8]:
n_clusters = 16
R_per_feature = []
@showprogress for j in  1:n_features
    R = kmeans(X_tr_vecs[[j],:], n_clusters; maxiter=200)
    push!(R_per_feature, R)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:01:32


In [9]:
P = hcat([r.centers[:] for r in R_per_feature]...);
println(P[:,1])
println(size(P))

Float32[99.267654, 5.2500305, 47.840622, 88.36083, 28.804195, 38.28978, 123.6862, 0.49666122, 12.244287, 111.930756, 20.320831, 64.371445, 79.39354, 136.85092, 56.391323, 71.9072]
(16, 128)


In [10]:
P = hcat([r.centers[:] for r in R_per_feature]...);
println(P[:,1])
println(size(P))

Float32[99.267654, 5.2500305, 47.840622, 88.36083, 28.804195, 38.28978, 123.6862, 0.49666122, 12.244287, 111.930756, 20.320831, 64.371445, 79.39354, 136.85092, 56.391323, 71.9072]
(16, 128)


In [11]:
@benchmark P = hcat([r.centers[:] for r in R_per_feature]...)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  11.625 μs … 43.334 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.916 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.280 μs ±  1.536 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▇█▁ ▂█▆▁                                                 
  ▂▂▂▅███▆████▇▆▆▆▆▅▅▃▃▃▃▃▃▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂ ▃
  11.6 μs         Histogram: frequency by time          19 μs <

 Memory estimate: 28.47 KiB, allocs estimate: 137.

Another way to construct the matrix would be

In [12]:
function build_prototype_matrix(R, n_clusters, n_features)
    
    prototypes = Array{Float32}(undef, n_clusters, n_features);
    for j in 1:n_features
        prototypes[:,j] .= vec(R_per_feature[j].centers)
    end
    return prototypes
end

build_prototype_matrix (generic function with 1 method)

In [13]:
@benchmark P = build_prototype_matrix(R_per_feature, n_clusters, n_features)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  57.333 μs … 162.583 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     60.833 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   61.130 μs ±   2.240 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

            ▁▃▅▆▆█▆▆▆▆▅▅█▄▄▃▃▃                                  
  ▁▂▂▂▃▃▄▅▅▇███████████████████▇█▆▅▅▅▄▄▄▃▂▂▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁ ▄
  57.3 μs         Histogram: frequency by time         67.7 μs <

 Memory estimate: 44.12 KiB, allocs estimate: 897.

In [14]:
P2 = build_prototype_matrix(R_per_feature, n_clusters, n_features);

In [531]:
isapprox(P, P2)

true

We can generate the PQcodes for the data as follows:

In [525]:
PQcodes = hcat([Int32.(r.assignments[:]) for r in R_per_feature]...)';

In [527]:
size(PQcodes)

(128, 1000000)

Here each vector prototype is in fact a single scalar (vecause subvectors have a single coordinate)

In [18]:
size(P)

(16, 128)

We need a method to, given a vector and a collection of prototypes per feature, find which are the closest

In [19]:
query = X_te_vecs[:,1];

In [20]:
closest_prototypes = Array{UInt8}(undef, n_features, 1);

In [21]:
typeof(typemax(Float32))

Float32

In [22]:
@inline function seuclidean(x::T,y::T) where {T}
    aux::T = (x -y)
    return aux * aux
end

function find_closest_coordinate(dist, 
                                 value::T,
                                 vector) where T
    best_coordinate = 1
    min_distance::T = typemax(T)
    for (j,x) in enumerate(vector)
        current_dist = dist(x, value)
        if current_dist < min_distance
            best_coordinate = j
            min_distance = current_dist
        end
    end
    return best_coordinate
end

find_closest_coordinate (generic function with 1 method)

In [23]:
find_closest_coordinate(seuclidean, Float32(48.), query)
print(query)

Float32[1.0, 3.0, 11.0, 110.0, 62.0, 22.0, 4.0, 0.0, 43.0, 21.0, 22.0, 18.0, 6.0, 28.0, 64.0, 9.0, 11.0, 1.0, 0.0, 0.0, 1.0, 40.0, 101.0, 21.0, 20.0, 2.0, 4.0, 2.0, 2.0, 9.0, 18.0, 35.0, 1.0, 1.0, 7.0, 25.0, 108.0, 116.0, 63.0, 2.0, 0.0, 0.0, 11.0, 74.0, 40.0, 101.0, 116.0, 3.0, 33.0, 1.0, 1.0, 11.0, 14.0, 18.0, 116.0, 116.0, 68.0, 12.0, 5.0, 4.0, 2.0, 2.0, 9.0, 102.0, 17.0, 3.0, 10.0, 18.0, 8.0, 15.0, 67.0, 63.0, 15.0, 0.0, 14.0, 116.0, 80.0, 0.0, 2.0, 22.0, 96.0, 37.0, 28.0, 88.0, 43.0, 1.0, 4.0, 18.0, 116.0, 51.0, 5.0, 11.0, 32.0, 14.0, 8.0, 23.0, 44.0, 17.0, 12.0, 9.0, 0.0, 0.0, 19.0, 37.0, 85.0, 18.0, 16.0, 104.0, 22.0, 6.0, 2.0, 26.0, 12.0, 58.0, 67.0, 82.0, 25.0, 12.0, 2.0, 2.0, 25.0, 18.0, 8.0, 2.0, 19.0, 42.0, 48.0, 11.0]

In [24]:
@benchmark find_closest_coordinate(seuclidean, Float32(48.), query)


BenchmarkTools.Trial: 10000 samples with 776 evaluations.
 Range (min … max):  162.532 ns … 325.709 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     162.747 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   164.051 ns ±   3.392 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▃▃▁            ▃▁    ▂▁      ▁                              ▁
  ███████▇█▇█▇██▇▇████▇█████████████▇▆▆▅▅▆▅▆▆▅▅▅▅▆▅▅▅▆▄▅▅▄▅▅▅▅▅ █
  163 ns        Histogram: log(frequency) by time        176 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [25]:
@assert 3 == find_closest_coordinate(seuclidean, 2., [3.,5,2,1])

@assert 4 == find_closest_coordinate(seuclidean, 1., [3.,5,2,1])

In [26]:
function encode(dist, vector, prototypes)
    closest_prototypes = Array{Int8}(undef, n_features, 1);
    @inbounds for (j,x) in enumerate(vector)
            closest_prototypes[j] = find_closest_coordinate(dist, x, prototypes[:,j])
    end
    return closest_prototypes
end

encode (generic function with 1 method)

In [27]:
find_closest_coordinate(sqeuclidean, query[1], P[:,1])

8

In [28]:
encode(sqeuclidean, query, P)

128×1 Matrix{Int8}:
  8
 16
  5
  3
 13
 12
  8
  7
  7
 11
 10
  4
 14
  ⋮
 11
 10
  2
  4
  8
  1
 11
  8
  9
  6
  3
 13

In [29]:
@benchmark encode(sqeuclidean, query, P)

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.030 μs …  12.721 ms  ┊ GC (min … max):  0.00% … 99.93%
 Time  (median):     5.137 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   6.503 μs ± 127.162 μs  ┊ GC (mean ± σ):  19.55% ±  1.00%

             ▂▆▆▅▅▅█▆▄▂                                        
  ▁▂▃▁▂▂▁▁▁▂▄████████████▅▆▅▅▄▃▃▂▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  4.03 μs         Histogram: frequency by time        7.74 μs <

 Memory estimate: 16.19 KiB, allocs estimate: 129.

In [30]:
function encode_fast(dist, vector::Array{T}, prototypes::Array{T}) where T
    n_clusters, n_features = size(prototypes)
    closest_prototypes = Array{Int8}(undef, n_features, 1);
    
    @inbounds for (j,x) in enumerate(vector)
        best_coordinate = 1
        min_distance::T = typemax(T)
        for k in 1:n_clusters
           current_dist = dist(prototypes[k,j],x )
           if current_dist < min_distance
               best_coordinate = k
               min_distance = current_dist
           end
           #println(k, ' ', j, ' ', best_coordinate, ' ',min_distance )
        end            
        closest_prototypes[j] = best_coordinate
    end
    return closest_prototypes
end

encode_fast (generic function with 1 method)

In [31]:
(query[1] .- P[:,1]) .* (query[1] .- P[:,1])

16-element Vector{Float32}:
  9656.532
    18.06276
  2194.044
  7631.915
   773.0733
  1390.5277
 15051.904
     0.25334996
   126.43398
 12305.633
   373.29453
  4015.94
  6145.547
 18455.473
  3068.1987
  5027.8315

In [32]:
println(find_closest_coordinate(sqeuclidean, query[1], P[:,1]))
println(find_closest_coordinate(sqeuclidean, query[2], P[:,2]))

8
16


In [33]:
encode_fast(sqeuclidean, query, P)[1:4]

4-element Vector{Int8}:
  8
 16
  5
  3

In [34]:
@assert isapprox(encode(sqeuclidean, query, P),
                 encode_fast(sqeuclidean, query, P))

In [35]:
@benchmark encode_fast(sqeuclidean, query, P)

BenchmarkTools.Trial: 10000 samples with 19 evaluations.
 Range (min … max):  953.947 ns …  5.230 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     984.684 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.005 μs ± 94.458 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

       █                                                        
  ▃▆▃▂▆█▆▆▃▂▂▂▂▃▃▃▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  954 ns          Histogram: frequency by time         1.24 μs <

 Memory estimate: 192 bytes, allocs estimate: 1.

In [36]:
function compute_ADC(query, prototypes, dist)
    n_clusters, n_features = size(P)
    
    ADC_table = Array{Float32}(undef, n_clusters, n_features)
    
    for j in 1:n_features
        for p in 1:n_clusters
            ADC_table[p,j] = dist(query[j], @view prototypes[p,:] )
        end
    end
    return  ADC_table
end

compute_ADC (generic function with 1 method)

In [37]:
size(P)

(16, 128)

In [38]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds   for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

SEuclidean0 (generic function with 1 method)

In [39]:
@benchmark compute_ADC(query, P, SEuclidean0)

BenchmarkTools.Trial: 7920 samples with 1 evaluation.
 Range (min … max):  586.000 μs … 95.076 ms  ┊ GC (min … max): 0.00% … 99.30%
 Time  (median):     613.041 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   630.068 μs ±  1.062 ms  ┊ GC (mean ± σ):  1.89% ±  1.12%

            ▂▃▇█▇▅▃▁                                            
  ▁▂▂▂▂▃▃▅▇█████████▇▅▅▅▅▅▄▄▄▄▃▃▂▃▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  586 μs          Histogram: frequency by time          692 μs <

 Memory estimate: 272.25 KiB, allocs estimate: 10501.

In [40]:
adc_table = compute_ADC(query, P, SEuclidean0);

In [41]:
function compute_ADC_fast(query, prototypes, dist)
    n_clusters, n_features = size(P)
    
    ADC_table = Array{Float32}(undef, n_clusters, n_features)
    
    for j in 1:n_features
        for p in 1:n_clusters
            
            ADC_table[p,j] = dist(query[j],  prototypes[p,:] )
        end
    end
    return  ADC_table
end

compute_ADC_fast (generic function with 1 method)

In [42]:
@benchmark compute_ADC_fast(query, P, SEuclidean0)

BenchmarkTools.Trial: 4789 samples with 1 evaluation.
 Range (min … max):  828.000 μs … 105.093 ms  ┊ GC (min … max):  0.00% … 99.04%
 Time  (median):     913.167 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.043 ms ±   3.478 ms  ┊ GC (mean ± σ):  11.80% ±  3.50%

                      ▂▆██▇▅▅▃▃▁                                 
  ▂▂▂▂▃▂▃▄▃▃▃▂▂▂▂▁▃▃▅▇████████████▇▆▆▅▅▄▄▄▄▄▄▄▃▃▃▃▂▃▃▃▂▃▂▃▂▂▃▂▂ ▄
  828 μs           Histogram: frequency by time         1.03 ms <

 Memory estimate: 1.48 MiB, allocs estimate: 18693.

In [43]:
function adc_dist(query_code, x_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @fastmath for j in eachindex(x_code)
        res+= adc_table[x_code[j], j]
    end
    return res
end

adc_dist (generic function with 1 method)

In [44]:
query_code = encode_fast(sqeuclidean, query, P)
x_code = PQcodes[:,1];
adc_table = compute_ADC(query, P, SEuclidean0);
adc_dist(query_code, x_code,  adc_table)

549694.4f0

In [45]:
@benchmark adc_dist(query_code, x_code,  adc_table)

BenchmarkTools.Trial: 10000 samples with 970 evaluations.
 Range (min … max):  77.835 ns … 146.865 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     80.068 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   80.770 ns ±   2.474 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁▃▃ ▃▃▁▄██▇▄▃▂▁▁▁▂▄▄▃▃▂▁▁ ▁▁▁                                ▂
  ███▇█████████████████████████████▇▆▇▆▆▅▆▅▆▅▆▅▅▆▅▃▅▃▄▅▃▅▅▄▄▄▄ █
  77.8 ns       Histogram: log(frequency) by time      91.7 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [46]:
x = X_tr_vecs[:,1];
@benchmark SEuclidean0(x, query)

BenchmarkTools.Trial: 10000 samples with 923 evaluations.
 Range (min … max):  112.495 ns … 235.193 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     114.166 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   115.594 ns ±   4.273 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂▆▁▃▇█▇▆▄▂▂▂▂▄▄▃▃▄▃▃▂▂▁▁▁▁▂▁▁▁▁▃▂▁ ▁                          ▂
  █████████████████████████████████████▇▇▆▇▇▆▇▆▆▆▅▆▅▄▅▅▆▅▄▅▄▅▄▅ █
  112 ns        Histogram: log(frequency) by time        129 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

##  Share prototypes across features

https://groups.google.com/g/julia-users/c/xBcQRebyi_o



In [543]:
n_clusters = 32
#R_shared = kmeans(Matrix(vec(X_tr_vecs)'), n_clusters; maxiter=200)
R_shared = kmeans(X_tr_vecs[[1],:], n_clusters; maxiter=200)

KmeansResult{Matrix{Float32}, Float32, Int64}(Float32[8.419607 137.42932 … 29.992058 40.49322], [16, 30, 16, 15, 16, 4, 16, 30, 1, 27  …  8, 1, 1, 26, 5, 29, 19, 31, 16, 7], Float32[0.05256829, 0.9406433, 0.05256829, 0.0007019043, 0.5940128, 0.0034179688, 0.05256829, 1.0611572, 0.17607117, 2.107788  …  0.2590332, 2.497635, 0.33685303, 0.0, 0.9626465, 2.2998047, 0.2913227, 0.00012207031, 0.05256829, 8.056641], [77320, 7852, 15690, 25974, 20854, 1463, 12399, 12794, 36379, 74528  …  12937, 17030, 11698, 22548, 41496, 11602, 16793, 41154, 24681, 12463], [77320, 7852, 15690, 25974, 20854, 1463, 12399, 12794, 36379, 74528  …  12937, 17030, 11698, 22548, 41496, 11602, 16793, 41154, 24681, 12463], 1.2748595f6, 6, true)

In [ ]:
function encode_shared(dist, vector::Array{T}, shared_prototypes::Array{T}) where T
    n_clusters = legnth(shared_prototypes)
    
    closest_prototypes = Array{Int8}(undef, n_features, 1);
    
    @inbounds for (j,x) in enumerate(vector)
        best_coordinate = 1
        min_distance::T = typemax(T)
        for k in 1:n_clusters
           current_dist = dist(prototypes[k,j],x )
           if current_dist < min_distance
               best_coordinate = k
               min_distance = current_dist
           end
           #println(k, ' ', j, ' ', best_coordinate, ' ',min_distance )
        end            
        closest_prototypes[j] = best_coordinate
    end
    return closest_prototypes
end

In [542]:
maximum(R_shared.assignments)

32

In [536]:
PQcodesShared = hcat([Int32.(r.assignments[:]) for r in R_shared]...)';

LoadError: MethodError: no method matching length(::KmeansResult{Matrix{Float32}, Float32, Int64})
[0mClosest candidates are:
[0m  length([91m::Union{Base.KeySet, Base.ValueIterator}[39m) at abstractdict.jl:58
[0m  length([91m::Union{ZMQ._Message, Base.RefValue{ZMQ._Message}}[39m) at ~/.julia/packages/ZMQ/R3wSD/src/_message.jl:31
[0m  length([91m::Union{HDF5.Attribute, HDF5.Dataset}[39m) at ~/.julia/packages/HDF5/T4H0V/src/dataspaces.jl:196
[0m  ...

In [48]:
length(unique(R_shared.centers))

32

In [49]:
println(maximum(R_shared.centers))
println(maximum(X_tr_vecs[[1],:]))

153.08621
172.0


In [50]:
P_shared = R_shared.centers

1×32 Matrix{Float32}:
 44.9633  100.415  2.90723  128.883  …  114.634  15.9686  124.474  26.4908

Now we need a new encoding function that uses a single vector of prototypes

In [51]:
function encode_fast_shared(dist, vector::Array{T}, prototypes::Array{T}) where T
    
    n_features = length(vector)
    n_clusters = length(prototypes)
    closest_prototypes = Array{Int8}(undef, n_features, 1);
    
    @inbounds @fastmath for (j,x) in enumerate(vector)
        best_coordinate = 1
        min_distance::T = typemax(T)
        for k in 1:n_clusters
           current_dist = dist(prototypes[k],x )
           if current_dist < min_distance
               best_coordinate = k
               min_distance = current_dist
           end
        end            
        closest_prototypes[j] = best_coordinate
    end
    return closest_prototypes
end

encode_fast_shared (generic function with 1 method)

In [52]:
@benchmark encode_fast_shared(sqeuclidean, query, P_shared)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.729 μs …  3.554 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.771 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.783 μs ± 86.371 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅█▄▃▂█▆▄▅▂▂▂▁▁▁▂▁▂▂▁▂▂ ▂▁                                  ▂
  █████████████████████████▇█▆▅▃▅▃▁▅▁▄▃▃▃▃▄▄▄▄▁▁▃▅▅▇█▆██▆██▇ █
  1.73 μs      Histogram: log(frequency) by time     2.14 μs <

 Memory estimate: 192 bytes, allocs estimate: 1.

In [53]:
function compute_ADC_shared(query, prototypes, dist)
    n_clusters = length(prototypes)
    ADC_table = Array{Float32}(undef, n_clusters)
    
    @inbounds  for p in 1:n_clusters            
        ADC_table[p] = dist(query[p],  prototypes[p] )
    end
    return  ADC_table
end

compute_ADC_shared (generic function with 1 method)

In [54]:
query_code = encode_fast_shared(sqeuclidean, query, P_shared)
x_code = PQcodes[:,1];
adc_table_shared = Float32.(compute_ADC_shared(query, P_shared, SEuclidean0));
#adc_dist(query_code, x_code,  adc_table)

In [55]:
@benchmark adc_table = compute_ADC_shared(query, P_shared, SEuclidean0)

BenchmarkTools.Trial: 10000 samples with 981 evaluations.
 Range (min … max):  63.710 ns …  97.715 μs  ┊ GC (min … max):  0.00% … 99.89%
 Time  (median):     73.989 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   84.853 ns ± 976.411 ns  ┊ GC (mean ± σ):  11.50% ±  1.00%

               ▁▇█▃▁▃▃▁                                         
  ▃▂▂▂▂▂▂▂▂▂▂▁▅█████████▇▇▆▅▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  63.7 ns         Histogram: frequency by time         96.9 ns <

 Memory estimate: 192 bytes, allocs estimate: 1.

## Improving distance between vectors with ADC_distance

In [207]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds   for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

SEuclidean0 (generic function with 1 method)

In [208]:
@benchmark SEuclidean0($x, $query)

BenchmarkTools.Trial: 10000 samples with 969 evaluations.
 Range (min … max):  77.442 ns … 178.277 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     77.615 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   79.027 ns ±   3.064 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▄▁      ▄▆▂▂       ▁▁                                       ▁
  ███▇▆▆▇▆▇█████▇▇▇▇▇▇██▇▇▆▇▇▅▅▆▆▆▅▅▅▅▄▅▄▅▄▃▃▅▃▃▂▅▅▄▅▅▄▄▅▄▅▄▂▄ █
  77.4 ns       Histogram: log(frequency) by time      92.4 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [339]:
function adc_dist_shared(query_code, x_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @simd for j in eachindex(x_code)
        res+= adc_table[x_code[j]]
    end
    return res
end

adc_dist_shared (generic function with 1 method)

In [340]:
@benchmark adc_dist_shared($query_code, $x_code, $adc_table_shared)

BenchmarkTools.Trial: 10000 samples with 992 evaluations.
 Range (min … max):  38.978 ns … 83.459 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     39.231 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   39.488 ns ±  1.096 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁▆▇██▄▃▄▁              ▁▂  ▁                               ▂
  ████████████▇▇█▆▇▇▇█▆▆███████▇▇▅▆▇▆▅▆▅▅▇▆▆▆▆▆▅▅▇▆▆▇▆▅█▅▆▇▆▆ █
  39 ns        Histogram: log(frequency) by time      42.3 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [329]:
function adc_dist_shared_unrolled(query_code, x_code,  adc_table)
    res1 = zero(eltype(adc_table))
    res2 = zero(eltype(adc_table))
    res3 = zero(eltype(adc_table))
    res4 = zero(eltype(adc_table))
    
    @inbounds @fastmath for j in 1:4:length(x_code)
        res1 += adc_table[x_code[j]]
        res2 += adc_table[x_code[j+1]]
        res3 += adc_table[x_code[j+2]]
        res4 += adc_table[x_code[j+3]]
    end
    
    return res1 + res2 + res3 + res4
end

adc_dist_shared_unrolled (generic function with 1 method)

In [330]:
@benchmark adc_dist_shared_unrolled($query_code, $x_code,  $adc_table_shared)

BenchmarkTools.Trial: 10000 samples with 995 evaluations.
 Range (min … max):  30.694 ns … 86.307 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     30.905 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   31.661 ns ±  2.603 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▁▁▅▄▂▁▃▃▂ ▁▁                                              ▂
  ██████████████▇▇▆▇▇▆▇▇▇▆▆▆▇▅▆▆▆▅▁▅▅▅▄▅▅▅▃▆▄▄▄▃▁▁▃▄▅▅▅▄▄▅▃▅▅ █
  30.7 ns      Histogram: log(frequency) by time      44.2 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [288]:
adc_dist_shared(query_code, x_code,  adc_table_shared)

430293.53f0

In [318]:
adc_dist_shared_unrolled(query_code, x_code,  adc_table_shared)

430293.47f0

Note that euclidean squared distance with `@fastmath` is still faster

In [212]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds @fastmath  for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

SEuclidean0 (generic function with 1 method)

In [213]:
@benchmark SEuclidean0($x, $query)

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):  9.510 ns … 22.480 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     9.635 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.771 ns ±  0.512 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁▅█ ▄▂▄ ▂ ▂▆ ▂▁▂ ▁ ▁                                      ▂
  ▃███▁███▁█▇██▁███▁████▁█▇▇▁█▇▆▆▁▆▆▇▁▆▆▄▅▁▆▅▄▁▄▅▄▄▁▅▄▃▁▅▄▅▄ █
  9.51 ns      Histogram: log(frequency) by time     11.4 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

What if we use an adctable cotaining only integers?

In [415]:
function adc_dist_shared(query_code, x_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @simd for j in eachindex(x_code)
        res+= adc_table[x_code[j]]
    end
    return res
end

adc_dist_shared (generic function with 1 method)

In [439]:
adc_table_shared_int = Int32.(round.(adc_table_shared));

In [431]:
@benchmark adc_dist_shared($query_code, $x_code,  $adc_table_shared_int)

BenchmarkTools.Trial: 10000 samples with 992 evaluations.
 Range (min … max):  37.341 ns … 111.350 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     37.676 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   38.597 ns ±   3.020 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆█▃▂▂▄▅▂▁▁▁▂▂▂▁                                              ▁
  ███████████████▇█▆█▇▇▇██▇▇▇▇▇▇▇▆▆▆▆▅▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▆▅▅▄▄▅▁▅ █
  37.3 ns       Histogram: log(frequency) by time      51.3 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

## BWcode

Since different the same value on a pqcode found in different coordinates, refers to the same real number  when using a SharedQuantizer, we can iterate over the 128 vector coordinates of a pqcode and add up the value sin the `adc_table_shared`  or.... we can iterate over a `K`-dimensional vector that we call the BW code and add the values in `adc_table_shared` weighted by the amount of each prototype has been seen in a pqcode, which is precisely what the BW code stores.

In [611]:
n_examples = size(X_tr_vecs,2)
BWcodes = Array{Int16}(undef, n_clusters, n_examples);
bw_code = pq_code_to_bw_code(x_code, n_clusters);

In [612]:
function pq_code_to_bw_code(pqcode, n_clusters)
    bw_code = zeros(eltype(pqcode), n_clusters)
    for c in pqcode
        bw_code[c] += 1
    end
    return bw_code
end

pq_code_to_bw_code (generic function with 1 method)

Now let´s verify that the distance between a query

In [609]:
function bw_adc_dist_shared(query_code, bw_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @simd for j in eachindex(bw_code)
        res+= adc_table[j] * bw_code[j]
    end
    return res
end

bw_adc_dist_shared (generic function with 1 method)

In [610]:
@benchmark bw_adc_dist_shared($query_code, $bw_code, $adc_table_shared)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  4.875 ns … 9.834 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.000 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.995 ns ± 0.085 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                    ▂         █                              
  ▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁█▆▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▃▃▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▂ ▂
  4.88 ns        Histogram: frequency by time       5.12 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [613]:
bw_adc_dist_shared(query_code, bw_code, adc_table_shared)

430293.53f0

In [614]:
adc_dist_shared(query_code, x_code, adc_table_shared)

430293.53f0

## Linear scann study

### Exact version

In [303]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds @fastmath  for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

function linear_scann_exact(query, X)

    n_features, n_examples = size(X)
    distances = Array{eltype(query)}(undef, n_examples)
    
    @inbounds @fastmath  for j in 1:n_examples
        distances[j] = SEuclidean0(query, X[:,j])    
    end
    return distances
end

@benchmark linear_scann_exact(query, X_tr_vecs)

BenchmarkTools.Trial: 27 samples with 1 evaluation.
 Range (min … max):  140.659 ms … 242.125 ms  ┊ GC (min … max):  0.00% … 41.35%
 Time  (median):     149.052 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   189.397 ms ±  47.933 ms  ┊ GC (mean ± σ):  24.96% ± 21.00%

   █▂▂                                                     ▂ ▅   
  ████▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅████ ▁
  141 ms           Histogram: frequency by time          242 ms <

 Memory estimate: 553.13 MiB, allocs estimate: 1000002.

In [297]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

function linear_scann_exact(query, X)

    n_features, n_examples = size(X)
    distances = Array{eltype(query)}(undef, n_examples)
    
    @inbounds @fastmath Threads.@threads for j in 1:n_examples
        distances[j] = SEuclidean0(query, X[:,j])    
    end
    return distances
end

@benchmark linear_scann_exact(query, X_tr_vecs)

BenchmarkTools.Trial: 58 samples with 1 evaluation.
 Range (min … max):  45.220 ms …    1.013 s  ┊ GC (min … max):  0.00% … 95.17%
 Time  (median):     49.153 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   98.816 ms ± 212.307 ms  ┊ GC (mean ± σ):  49.78% ± 21.25%

  █                                                             
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▄ ▁
  45.2 ms       Histogram: log(frequency) by time       1.01 s <

 Memory estimate: 553.14 MiB, allocs estimate: 1000064.

In [471]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds @fastmath  for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

function linear_scann_exact(query, X)

    n_features, n_examples = size(X)
    distances = Array{eltype(query)}(undef, n_examples)
    
    @inbounds for j in 1:n_examples
        distances[j] = SEuclidean0(query, view(X,:,j))    
    end
    return distances
end

@benchmark linear_scann_exact(query, X_tr_vecs)

BenchmarkTools.Trial: 488 samples with 1 evaluation.
 Range (min … max):   9.681 ms … 36.988 ms  ┊ GC (min … max): 0.00% … 72.23%
 Time  (median):     10.143 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.247 ms ±  1.226 ms  ┊ GC (mean ± σ):  0.53% ±  3.27%

                    ▃█▅▄▂            ▂                         
  ▂▂▂▂▂▂▄▂▃▂▂▃▃▃▃▃▃▄█████▅▅▃▄▃▅▅▆▆▇███▇▇▅▅▃▄▃▁▃▁▁▁▁▁▁▂▂▁▁▁▁▁▂ ▃
  9.68 ms         Histogram: frequency by time        10.8 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 2.

### ADC version

In [470]:
function linear_scann(query, PQcodes, adc_table_shared, P_shared)
    
    query_code = encode_fast_shared(sqeuclidean, query, P_shared)
    n_features, n_examples = size(PQcodes)
    distances = Array{eltype(query)}(undef, n_examples)
    
    @inbounds @fastmath for j in 1:n_examples
        distances[j] = adc_dist_shared(query_code,  view(PQcodes,:,j) ,  adc_table_shared)    
    end
    return distances
end

@benchmark distances = linear_scann($query, $PQcodes, $adc_table_shared, $P_shared)

BenchmarkTools.Trial: 63 samples with 1 evaluation.
 Range (min … max):  79.867 ms …  83.343 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     80.252 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   80.596 ms ± 823.068 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁▁▆▁ ▁█▆▆▁     ▁                                              
  ████▇█████▄▇▄▇▁█▁▁▁▄▄▄▁▁▄▁▄▄▁▁▁▁▁▁▁▄▁▁▁▄▁▁▁▁▄▁▁▄▁▄▁▁▄▁▄▁▁▁▁▇ ▁
  79.9 ms         Histogram: frequency by time         82.7 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 3.

In [291]:
function linear_scann(query, PQcodes, adc_table_shared, P_shared)
    
    query_code = encode_fast_shared(sqeuclidean, query, P_shared)
    n_features, n_examples = size(PQcodes)
    distances = Array{eltype(query)}(undef, n_examples)
    
    @inbounds @fastmath Threads.@threads for j in 1:n_examples
        distances[j] = adc_dist_shared(query_code,  PQcodes[:,j] ,  adc_table_shared)    
    end
    return distances
end

BenchmarkTools.Trial: 126 samples with 1 evaluation.
 Range (min … max):  23.466 ms … 881.834 ms  ┊ GC (min … max):  0.00% … 96.80%
 Time  (median):     25.155 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   39.815 ms ± 107.178 ms  ┊ GC (mean ± σ):  33.96% ± 12.15%

  █                                                             
  █▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▄
  23.5 ms       Histogram: log(frequency) by time       878 ms <

 Memory estimate: 186.93 MiB, allocs estimate: 1000064.

In [294]:
function linear_scann_unrolled(query, PQcodes, adc_table_shared, P_shared)
    
    query_code = encode_fast_shared(sqeuclidean, query, P_shared)
    n_features, n_examples = size(PQcodes)
    distances = Array{eltype(query)}(undef, n_examples)
    
    @inbounds @fastmath Threads.@threads for j in 1:n_examples
        distances[j] = adc_dist_shared_unrolled(query_code,  PQcodes[:,j] ,  adc_table_shared)    
    end
    return distances
end

@benchmark distances = linear_scann_unrolled($query, $PQcodes, $adc_table_shared, $P_shared)

BenchmarkTools.Trial: 138 samples with 1 evaluation.
 Range (min … max):  22.545 ms … 847.083 ms  ┊ GC (min … max):  0.00% … 97.26%
 Time  (median):     23.738 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   36.216 ms ±  98.161 ms  ┊ GC (mean ± σ):  32.78% ± 11.67%

  █                                                             
  █▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▄
  22.5 ms       Histogram: log(frequency) by time       837 ms <

 Memory estimate: 186.93 MiB, allocs estimate: 1000064.

### ADC version with BW codes

In [627]:
n_examples = size(X_tr_vecs,2)
BWcodes = Array{Int8}(undef, n_clusters, n_examples);
bw_code = pq_code_to_bw_code(x_code, n_clusters);

In [628]:

for j in 1:n_examples
   BWcodes[:,j] .= pq_code_to_bw_code(PQcodes[:,j], n_clusters)
end

In [691]:
function bw_adc_dist_shared(query_code, bw_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @simd for j in eachindex(bw_code)
        res+= adc_table[j] * bw_code[j]
    end
    return res
end

function linear_scann_bw(query, BWcodes, adc_table_shared, P_shared)
    
    query_code = encode_fast_shared(sqeuclidean, query, P_shared)
    n_features, n_examples = size(BWcodes)
    distances = Array{eltype(query)}(undef, n_examples)
    n_clusters = size(BWcodes,1)
    #@show n_clusters
    @inbounds for i in 1:n_examples
        
        # Following code is equivalent to bw_adc_dist_shared(query_code, BWcodes[:,i],  adc_table_shared)
        res = zero(eltype(adc_table_shared))
        @simd for j in 1:n_clusters
            res+= adc_table_shared[j] * BWcodes[j,i]
        end        
        distances[i] = res  
    
    end
    return distances
end

#@benchmark distances = linear_scann($query, $PQcodes, $adc_table_shared, $P_shared)

linear_scann_bw (generic function with 1 method)

In [694]:
@benchmark linear_scann_bw($query, $BWcodes, $adc_table_shared, $P_shared)

BenchmarkTools.Trial: 1078 samples with 1 evaluation.
 Range (min … max):  3.622 ms …   6.427 ms  ┊ GC (min … max): 0.00% … 15.38%
 Time  (median):     4.494 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.629 ms ± 432.548 μs  ┊ GC (mean ± σ):  1.60% ±  4.52%

                 ▅█▇▇▂▃ ▂ ▂                                    
  ▂▁▂▁▁▂▃▂▁▃▂▂▂▃▅████████████▅▅▅▄▃▅▆▅▃▅█▆▄▅▄▅▃▃▂▃▄▃▃▂▂▃▃▂▂▂▃▃ ▄
  3.62 ms         Histogram: frequency by time        5.89 ms <

 Memory estimate: 3.82 MiB, allocs estimate: 64.

#### TODO
How can we leverage a fast multithreaded Linearscann?


In [699]:
BWCodes

LoadError: UndefVarError: BWCode not defined

In [708]:
function bw_adc_dist_shared(query_code, bw_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @simd for j in eachindex(bw_code)
        res+= adc_table[j] * bw_code[j]
    end
    return res
end

function linear_scann_bw_multithreaded(query, BWcodes, adc_table_shared, P_shared)
    
    query_code = encode_fast_shared(sqeuclidean, query, P_shared)
    n_features, n_examples = size(BWcodes)
    distances = Array{eltype(query)}(undef, n_examples)
    n_clusters = size(BWcodes,1)
    #@show n_clusters
    @inbounds @fastmath Threads.@threads  for i in 1:n_examples
       distances[i] = bw_adc_dist_shared(query_code, view(BWcodes,:,i),  adc_table_shared)
    end    
    
    return distances
end

@benchmark distances = linear_scann_bw_multithreaded($query, $BWcodes, $adc_table_shared, $P_shared)

BenchmarkTools.Trial: 4774 samples with 1 evaluation.
 Range (min … max):  674.583 μs … 87.179 ms  ┊ GC (min … max):  0.00% … 98.88%
 Time  (median):     858.750 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.037 ms ±  1.377 ms  ┊ GC (mean ± σ):  12.79% ± 16.03%

   ▄▆██▇▆▅▃▂▂▁▁▁▁                                              ▁
  ▄███████████████▇█▇▇▆▆▅▆▅▇▇▇▇▇███▇██▇▇▇▇▅▇▇▆▆▇▆▆▇▆▅▄▄▃▄▄▄▅▃▅ █
  675 μs        Histogram: log(frequency) by time      2.93 ms <

 Memory estimate: 3.82 MiB, allocs estimate: 64.

In [709]:
distances = linear_scann_bw_multithreaded(query, BWcodes, adc_table_shared, P_shared)
distances

1000000-element Vector{Float32}:
 430293.53
 409572.12
 523334.03
 411390.56
 483739.44
 465726.72
 469184.3
 465421.94
 471216.94
 472431.4
 505966.56
 441302.97
 504491.28
      ⋮
 353777.56
 407598.38
 390484.38
 529897.25
 490457.53
 362624.6
 516215.8
 429939.62
 412462.94
 407520.47
 499928.0
 393646.03

# Quality verification

In [689]:
length(linear_scann_bw(query, BWcodes, adc_table_shared, P_shared))

1000000

## Euclidean  Squared Distance

In [440]:
function SEuclidean(X, query)
    d = (query .- X) .* (query .- X)
    return sum(d, dims=1)
end

SEuclidean (generic function with 1 method)

In [441]:
function SEuclidean_2(X, query)
    n_features, n_examples = size(X)
    result = zeros(n_examples)
    for m in 1:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        result[m] = res
    end
    return result
end

SEuclidean_2 (generic function with 1 method)

In [442]:
query = X_te_vecs[:,1];

In [443]:
@benchmark SEuclidean(X_te_vecs, query)

BenchmarkTools.Trial: 3753 samples with 1 evaluation.
 Range (min … max):  928.708 μs … 32.129 ms  ┊ GC (min … max): 0.00% … 96.23%
 Time  (median):       1.211 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.328 ms ±  1.582 ms  ┊ GC (mean ± σ):  7.66% ±  6.17%

                      ▅▇██▇▆▆▅▅▅▄▂     ▂▃▃▂▂▁▁                  
  ▅▅▄▃▂▂▂▂▂▂▂▂▁▂▂▂▂▂▄▇██████████████▇▇████████▅▅▅▄▄▃▃▃▃▃▃▂▃▂▂▂ ▅
  929 μs          Histogram: frequency by time          1.5 ms <

 Memory estimate: 4.92 MiB, allocs estimate: 4.

In [444]:
@benchmark SEuclidean_2(X_te_vecs, query)

BenchmarkTools.Trial: 6369 samples with 1 evaluation.
 Range (min … max):  769.583 μs …  2.304 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     780.625 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   783.792 μs ± 22.694 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▂▂▄▅▆█▆▇▆▃▁                                               
  ▃▃▃████████████▇▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  770 μs          Histogram: frequency by time          831 μs <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

In [445]:
@benchmark SEuclidean(X_tr_vecs, query)

BenchmarkTools.Trial: 39 samples with 1 evaluation.
 Range (min … max):  115.288 ms … 213.858 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     121.950 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   131.411 ms ±  21.901 ms  ┊ GC (mean ± σ):  5.18% ± 6.91%

  ▄█           ▁                                                 
  ██▇▆▄▆▄▄▁▁▁▁▄█▇▆▁▄▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▄ ▁
  115 ms           Histogram: frequency by time          214 ms <

 Memory estimate: 492.10 MiB, allocs estimate: 4.

In [446]:
@benchmark SEuclidean_2(X_tr_vecs, query)

BenchmarkTools.Trial: 65 samples with 1 evaluation.
 Range (min … max):  77.253 ms …  79.056 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     77.805 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   77.872 ms ± 390.832 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▃▆█ ▆         █    ▁                                 
  ▇▁▁▁▁▁▄▁▇███▇█▄▄▁▇▄▇▄▇▁█▇▄▇▁█▄▁▁▁▄▁▁▄▁▇▄▄▄▄▁▄▁▁▁▄▁▁▁▁▁▄▁▁▁▁▄ ▁
  77.3 ms         Histogram: frequency by time         78.9 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

## Finding top k distances (and their ids)


The first naive thing we can do consist on computing all distances and then sorting them to get the top k closest vectors to the query vector

In [447]:
function top_k_ids(X, query)
    distances = SEuclidean_2(X, query)
    top_k_indices = sortperm(distances)
    return top_k_indices
end

top_k_ids (generic function with 1 method)

In [448]:
@benchmark top_k_ids(X_te_vecs, query)[1:10]

BenchmarkTools.Trial: 3988 samples with 1 evaluation.
 Range (min … max):  1.228 ms …  1.796 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.247 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.252 ms ± 20.688 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▁▅▅▇█▅▆▃▂                                              
  ▂▃▃▅▆███████████▇▅▅▅▄▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  1.23 ms        Histogram: frequency by time        1.32 ms <

 Memory estimate: 156.50 KiB, allocs estimate: 6.

A slightly better approach consist on using `partialsortperm` to simply sort a subset of the distances vector.

In [449]:
function top_k_ids_2(X, query, k)
    distances = SEuclidean_2(X, query)
    top_k_indices = partialsortperm(distances, 1:k)
    return top_k_indices
end

top_k_ids_2 (generic function with 1 method)

In [450]:
@benchmark top_k_ids_2(X_te_vecs, query, 10)

BenchmarkTools.Trial: 6056 samples with 1 evaluation.
 Range (min … max):  785.709 μs …  1.346 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     808.042 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   822.902 μs ± 37.672 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▁▇█▄▁                                                      
  ▃▃▃█████▇▆▅▄▄▃▃▄▄▄▄▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  786 μs          Histogram: frequency by time          944 μs <

 Memory estimate: 156.44 KiB, allocs estimate: 7.

### Storing top k distances in a priority queue

A better alternative consists on using a priority queue. This. queue will keep only k distances in memory (no need to store all distances between the query point and all possible candidates).

In [451]:
sort!([1,54,3,24,10])

5-element Vector{Int64}:
  1
  3
 10
 24
 54

In [452]:
a = [1,2,3,4,5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [453]:
function SEuclidean_3(X, query, top_k)
    result = sort(SEuclidean_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        dist = res/n_features
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end
            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end

    end
    return result
end

SEuclidean_3 (generic function with 1 method)

In [454]:
@benchmark SEuclidean_3(X_tr_vecs, query, 10) 

BenchmarkTools.Trial: 62 samples with 1 evaluation.
 Range (min … max):  79.984 ms … 90.506 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     80.591 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   81.169 ms ±  1.771 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

       █▂                                                      
  ▆▇▇▆▇██▁▅▁▄▁▃▁▃▃▁▃▃▁▃▃▁▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▃▁▁▁▁▁▁▁▃▁▁▁▃ ▁
  80 ms           Histogram: frequency by time        86.2 ms <

 Memory estimate: 16.67 KiB, allocs estimate: 120.

In [455]:
@benchmark top_k_ids_2(X_tr_vecs, query, 10)

BenchmarkTools.Trial: 54 samples with 1 evaluation.
 Range (min … max):  86.799 ms … 150.780 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     89.566 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   93.604 ms ±  11.673 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃ ▃█                                                         
  ▄████▇▄▁▃▁▁▁▄▄▁▁▁▁▁▃▁▁▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▁▁▁▃ ▁
  86.8 ms         Histogram: frequency by time          125 ms <

 Memory estimate: 15.26 MiB, allocs estimate: 7.

In [456]:
function SEuclidean_4(X, query, top_k)
    
    result = sort(SEuclidean_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    @inbounds @fastmath for m in top_k:n_examples
        res = zero(eltype(X))
        @simd for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        dist = res
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end


SEuclidean_4 (generic function with 1 method)

In [462]:
@benchmark SEuclidean_4(X_tr_vecs, query, 10) 

BenchmarkTools.Trial: 416 samples with 1 evaluation.
 Range (min … max):  11.872 ms …  13.198 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.996 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.029 ms ± 161.525 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█              ▁  ▁                                          
  ███▆▅▆▅▄▃▅▃▄▃▃▃▃██▇█▅▄▂▃▃▃▂▃▁▂▁▁▁▂▁▂▃▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂ ▃
  11.9 ms         Histogram: frequency by time         12.7 ms <

 Memory estimate: 15.97 KiB, allocs estimate: 114.

In [458]:
X_tr_200k = X_tr_vecs[:,1:200_000]
@benchmark SEuclidean_4(X_tr_200k, query, 10) 

BenchmarkTools.Trial: 2022 samples with 1 evaluation.
 Range (min … max):  2.424 ms …  3.359 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.448 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.463 ms ± 52.802 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃█▅▄▁                                                       
  ██████▇▇▆▆▅▆▆▆▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▂▂▂▂▂▂▁▁▂▂ ▃
  2.42 ms        Histogram: frequency by time        2.68 ms <

 Memory estimate: 13.94 KiB, allocs estimate: 94.

In [ ]:
SEuclidean_4(X_tr_vecs, query, 10)

In [459]:
function SEuclidean_5(X, query, top_k)
    
    result = sort(SEuclidean_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        res = zero(eltype(X))
        @turbo  for j in 1:n_features
            aux = (query[j] - X[j,m])
            res += aux * aux
        end
        dist = res
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end

SEuclidean_5 (generic function with 1 method)

In [460]:
@benchmark SEuclidean_5(X_tr_200k, query, 10) 

BenchmarkTools.Trial: 2251 samples with 1 evaluation.
 Range (min … max):  2.148 ms …  2.755 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.197 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.211 ms ± 58.715 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▄▇██▇▆▆▇▇▆▅▄▃▂▂▁▁                                        ▂
  ▄▄██████████████████▆█▆▆▇▇▆▇▇▆█▇▆▇▅▅▅▆▄▁▄▅▁▄▁▅▄▄▅▁▄▅▁▅▅▅▁▆ █
  2.15 ms      Histogram: log(frequency) by time     2.52 ms <

 Memory estimate: 13.94 KiB, allocs estimate: 94.

# Comparison with Distances.jl


Distances.jl provides the method 'pairwise' to compute pairwise distances between two matrices.

In [ ]:
@show typeof(query)
@show size(query)
@show typeof(X_tr_vecs)
@show size(X_tr_vecs)

In [ ]:
v1 = [1,2,3]
v2 = [0,0,1]

y = [0,0,2]
X = [v1 v2]

In [ ]:
function SEuclidean_5(X, query, top_k)
    
    result = sort(SEuclidean_5(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        dist = zero(eltype(X))
        @turbo  for j in 1:n_features
            dist = (query[j] - X[j,m])
            dist += aux * aux
        end
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end

In [ ]:
function SEuclidean_2(X, query)
    n_features, n_examples = size(X)
    result = zeros(eltype(X), n_examples)
    for m in 1:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        result[m] = res
    end
    return result
end

In [ ]:
y_colvec = [y y][:,[1]]
query_mat = [query query][:,[1]];

In [ ]:
SEuclidean_2(X, y)

In [ ]:
pairwise(SqEuclidean(), y_colvec, X)

In [ ]:
@benchmark pairwise(SqEuclidean(), query_mat, X_tr_vecs)

In [ ]:
@benchmark SEuclidean_2(X_tr_vecs, query)

In [ ]:
@benchmark SEuclidean_4(X_tr_vecs, query, 10)

We can verify that the sorted distances by `SEuclidean_4` are the same as the results provided with `pairwise` from `Distances.jl`

In [ ]:
res2 = SEuclidean_4(X_tr_vecs, query, 10);
res = pairwise(SqEuclidean(), query_mat, X_tr_vecs)
res = sort(res, dims=2)[1:10]
res == res2

## Parallel implementation

# SIMD tests

In [ ]:

function find_val_in_array_simd(x::Array{T}, val::T) where {T}
    n_simd = 64
    last_pos_simd_chunk = length(x)-n_simd
    @inbounds for i in 1:n_simd:last_pos_simd_chunk
        vec_i = vload(Vec{n_simd, T}, x, i)
        sum_equality = sum(vec_i == val)
        if sum_equality >0
            return true
        end
    end

    @inbounds for i in last_pos_simd_chunk:length(x)
        if x[i] == val
            return true
        end
    end

    return false
end

In [ ]:
function jdotavx(a, b)
    s = zero(eltype(a))
    @turbo for i ∈ eachindex(a, b)
        s += a[i] * b[i]
    end
    s
end